##  Training Dataset Preparation

Input to model : 8 sec of audio clip with My sound Dubbed in between . 

Label : Label some number of time steps 

Inference : Model listens for 8 second for a while. Then, processes it . waits for a while. Then, Processes ..

More like, multithread, such that, by time it processes for prediction generation, it's preparing next wave .

The computaiton control is done by time.sleep .. 

In [1]:
import numpy as np
from pydub import AudioSegment
import random
import sys
import io
import os
import glob
import IPython
%matplotlib inline

In [2]:
import logging

In [3]:
os.chdir('../')

In [4]:
# IPython.display.Audio("data/external/sample_internet.wav")

In [5]:
ls

 Volume in drive C is OS
 Volume Serial Number is 7095-B01C

 Directory of C:\Users\Robin\Downloads\Podcast-Audio-Processing

03/27/2022  02:15 PM    <DIR>          .
03/27/2022  02:15 PM    <DIR>          ..
03/27/2022  01:06 PM    <DIR>          .git
03/27/2022  01:06 PM                80 .gitignore
03/27/2022  01:03 PM    <DIR>          .ipynb_checkpoints
03/27/2022  01:06 PM         5,854,438 audio_augmentation.ipynb
03/27/2022  01:53 PM        30,014,677 back_sover_ind_chap_1.mp3
03/27/2022  07:26 PM    <DIR>          data
03/27/2022  01:12 PM                 0 Design_.md
03/27/2022  01:06 PM             7,589 feature_extraction.py
03/27/2022  11:17 PM    <DIR>          notebooks
03/27/2022  01:06 PM             1,222 plot_helper.py
03/27/2022  12:51 PM             3,214 README.md
03/27/2022  01:06 PM    <DIR>          src
               7 File(s)     35,881,220 bytes
               7 Dir(s)  105,552,412,672 bytes free


In [6]:
pwd

'C:\\Users\\Robin\\Downloads\\Podcast-Audio-Processing'

In [7]:
from pydub import AudioSegment
from pathlib import Path

song1_part1_path = Path("./data/Good_Backgrounds/back_bitcooin_book_p1.wav")
song1_part2_path = Path("./data/Good_Backgrounds/background_lex_recorded.wav")
song2_path = Path('data/Good_Backgrounds/The Sovereign Individual-Part01-1.mp3')
# ./data/The Sovereign Individual-Part01-1.mp3')
# data/The Sovereign Individual-Part01-1.mp3
# data/Saifedean A. - The Bitcoin Standard.mp3
song1_part1 = AudioSegment.from_wav(song1_part1_path)
song1_part2 = AudioSegment.from_wav(song1_part2_path)
song2 = AudioSegment.from_mp3(song2_path)

In [8]:
## Oversample fromduration_secondsans part
song = song1_part1 + song2 + (song1_part2 * 2) 
del song1_part1 
del song1_part2
del song2

In [9]:
print('Total time (minutes) of Background Audio : ', song.duration_seconds // 60)

Total time (minutes) of Background Audio :  314.0


In [10]:
def get_clips_from_folder(root_):
    '''
    Get all the audio snippets from a folder
    '''
    clips = []
    for i in os.listdir(root_):
        if i[0].isdigit():
            
            assert i.endswith('.wav'), \
                "Make Sure only wav files are present in the folder"
            clip_path = os.path.join( root_ , i ) 
            clip = AudioSegment.from_wav(clip_path)
            print(f'Clip Vol : {clip.dBFS} , Clip Len {clip.duration_seconds}' )
            clips.append( clip )
    return clips

root_folder_start =  Path('./data/voices/start/')
clips_start = get_clips_from_folder(root_folder_start)

root_folder_pause =  Path('./data/voices/pause/')
clips_pause = get_clips_from_folder(root_folder_pause)

Clip Vol : -28.09047385386595 , Clip Len 0.7082086167800453
Clip Vol : -28.849303806635746 , Clip Len 0.905578231292517
Clip Vol : -23.943557094961818 , Clip Len 0.8359183673469388
Clip Vol : -24.886112340212794 , Clip Len 0.7198185941043084
Clip Vol : -26.758268700597117 , Clip Len 0.9520181405895691
Clip Vol : -27.257117140644958 , Clip Len 1.0913378684807256
Clip Vol : -24.137710427969576 , Clip Len 0.7546485260770975
Clip Vol : -24.970260476010896 , Clip Len 0.5456689342403628
Clip Vol : -29.762006543699226 , Clip Len 1.242267573696145
Clip Vol : -28.24661362828008 , Clip Len 0.626938775510204
Clip Vol : -21.91979724347295 , Clip Len 0.626938775510204
Clip Vol : -29.13288896567622 , Clip Len 1.207437641723356
Clip Vol : -28.24661362828008 , Clip Len 0.7314285714285714
Clip Vol : -26.16708789080999 , Clip Len 0.7546485260770975
Clip Vol : -24.55296410058984 , Clip Len 0.7082086167800453
Clip Vol : -26.530429023977288 , Clip Len 0.6965986394557823
Clip Vol : -21.681723416014613 , Cli

In [11]:
from pydub.playback import play
import random

## Play a snippet of background

In [12]:
n = 1000
play( song[5345 * n : 5349 * n] ) 

In [111]:
## experimental 
def adjust_level(sound, deviation=None , default = True):
    if default or deviation is None:
        return sound
    else:
        difference = np.random.normal(clip.dBFS , deviation )
        # print(f'Original : {clip.dBFS}')
        # print(f'Deviated by : {difference}')
        return sound.apply_gain(difference)

def sample_from_background( background, clip_duration_to_extract = 4,  multiplier = 1000 ):
    '''
    Sample a snippet of clip from supplied Audio
    
    Args:
        background: `pydub.audio_segment.AudioSegment`
            Audio clip from which a snippet is to be sampled
        clip_duration_to_extract: `int`
            Time (in seconds) of the extracted clip
        multiplier: `int`
            To work in seconds instead of milliseconds
            
    Returns: 
        A clip of length `clip_duration_to_extract`
    '''
    total_duration_ = background.duration_seconds - clip_duration_to_extract
    selection_start = np.random.uniform(  0 , total_duration_  ) 
    selection_end = (selection_start + clip_duration_to_extract) * multiplier
        
    background_clip = background[ selection_start * multiplier : selection_end  ] 
    return background_clip

def overlay_clip_to_bg( background , clip ):
    '''
    Dub Clip to background
    '''
    overlay_point_start = np.random.uniform( 0 , background.duration_seconds ) * 1000
    overlay_point_end = overlay_point_start + len(clip)
    overlay_point_end = min( len(background) , overlay_point_end  )
    
    return background.overlay( clip , position =  overlay_point_start ) , overlay_point_start , overlay_point_end

In [112]:
## sanity checks
clip_duration = 2.5
_sampled_background_clip =  sample_from_background(song, clip_duration_to_extract=clip_duration)
assert _sampled_background_clip.duration_seconds == clip_duration

In [113]:
# %%time
# ## check the goddamn speed
# for i in range(2000):
#     _ = sample_from_background(song)

## Get Clips with me Speaking something else
Otherwise model may cheat, and only learn to predict when it hears the overdubbed audio

In [114]:
negative_audio_path = Path('./data/voices/negative_examples.wav')
negative_audio = AudioSegment.from_wav(negative_audio_path)

# get snippets of negative clips between duration of 0.3 seconds to 1.8 seconds
clips_negative = [ sample_from_background( 
                        negative_audio,  
                        random.uniform(0.3, 2.2 )
                    ) for i in range(200) ]

In [115]:
clips_negative[24]

In [116]:
def generate_single_example( background_sample , clip_to_dub, background_only=False ):
    
    ## positive labels here
    if not background_only:
        overlayed_clip, time_stamp_start , time_stamp_end = overlay_clip_to_bg( background_sample, clip_to_dub )  
        return overlayed_clip , time_stamp_start , time_stamp_end
    
    else:
        ## Background label has No any sound, or some of my own sounds. .. 
        if random.random() >= 0.5:
            overlayed_clip, time_stamp_start , time_stamp_end = overlay_clip_to_bg( background_sample, clip_to_dub )  
            return overlayed_clip , time_stamp_start , time_stamp_end
        else:
            return background_sample , None, None

In [123]:
def volume_adjust(a,b,adjust_type='diff'):
    '''
    a : background clip
    b : l, l
    '''
    adjust_val = max(a.dBFS, b.dBFS) - min(a.dBFS, b.dBFS)
    # print(a.dBFS, b.dBFS)
    # print(adjust_val)
    if adjust_type == 'diff':
        if a.dBFS > b.dBFS:
            print('inside a greater')
            return a - adjust_val, b 
        
        else :
            print('inside b')
            return a, b-adjust_val
    else:
        mean_diff = (a.dBFS + b.dBFS ) / 2
        if a.dBFS > b.dBFS:
            mean_adjust = a.dBFS - mean_diff
            return (a - mean_adjust, b + mean_adjust)
        else :
            mean_adjust = b.dBFS - mean_diff
            return (a + mean_adjust, b - mean_adjust)
        
        
def extra_boost_clip(clip):
    return clip + random.choice([0,1,2,3,4])
            
        # return (a - mean_diff, b + mean_diff) if a.dBFS > b.dBFS else (a + mean_diff, b- mean_diff)

In [124]:
background_sample = sample_from_background( song )
clip_to_dub = random.choice( clips_start )


# print(background_sample.max_dBFS, clip_to_dub.max_dBFS)

adjusted_back, adjusted_clip = volume_adjust( background_sample ,  clip_to_dub,adjust_type='dsf' )
adjusted_clip = extra_boost_clip(adjusted_clip)
print(adjusted_back.dBFS, adjusted_clip.dBFS)



-24.214880734939047 -22.2123243667956


In [125]:
generate_single_example(adjusted_back, adjusted_clip)[0]
# generate_single_example(background_sample, clip_to_dub)[0]


## Now Generate Training Data

In [126]:
save_path = Path("./data/generated/")
save_path.mkdir(parents=True, exist_ok=True)

In [127]:
import pandas as pd
def prep_examples( no_of_examples = 3000 ):
    meta_data = pd.DataFrame(columns=['filename','start_time','end_time','label_type'])
        
    for i in range( no_of_examples ):
        save_name = f'{i}.wav'
        
        
        sample_song = sample_from_background( song )
        

        prob = random.random()
        if prob < 0.333333 : 
            sample_clip = random.choice(clips_start)
            sample_song, sample_clip = volume_adjust( sample_song ,   sample_clip   , adjust_type='dsf' )
            sample_clip = extra_boost_clip(sample_clip)
            sample_output , s ,e = generate_single_example( sample_song  , sample_clip )
                    
            label_type = 'start'
        elif prob >= 0.33 and prob <= 0.66666:
            sample_clip = random.choice(clips_pause)
            sample_song, sample_clip = volume_adjust( sample_song ,   sample_clip   , adjust_type='dsf' )
            sample_clip = extra_boost_clip(sample_clip)
            sample_output , s ,e = generate_single_example( sample_song  , sample_clip )
        
            label_type = 'pause'
        else:
            sample_clip = random.choice(clips_negative)
            sample_song, sample_clip = volume_adjust( sample_song ,   sample_clip   , adjust_type='dsf' )
            sample_clip = extra_boost_clip(sample_clip)
            sample_output , s ,e = generate_single_example( sample_song  , sample_clip, background_only=True )
            label_type = 'None'
        
        
        
        sample_output.export( save_path / save_name ,format='wav')
        meta_data.loc[i] = [ save_name, s, e, label_type   ] 
    
    return meta_data

In [128]:
meta = prep_examples(3000)

In [129]:
meta.label_type.value_counts()

pause    1070
None      995
start     935
Name: label_type, dtype: int64

In [130]:
meta.to_csv('data//meta_data.csv',index=False)

In [132]:
meta[40:50]

,filename,start_time,end_time,label_type
40,40.wav,1544.300768,2252.300768,pause
41,41.wav,NaN,NaN,None
42,42.wav,NaN,NaN,None
43,43.wav,3525.133487,4000.000000,start
44,44.wav,231.411710,1196.411710,None
45,45.wav,2628.531348,3719.531348,start
46,46.wav,1455.904137,2024.904137,pause
47,47.wav,1582.851710,2255.851710,pause
48,48.wav,2964.512177,4000.000000,start
49,49.wav,1114.762865,2356.762865,start


In [27]:
# save_path = "data\\external\\processed\\"
# train_path = os.path.join(save_path, 'train')
# test_path = os.path.join(save_path, 'test')

# os.makedirs(train_path, exist_ok=True)
# os.makedirs(test_path, exist_ok=True)

In [28]:
# import shutil
# for ind, files in enumerate(os.listdir(save_path)):
#     try:
#         int(files[0])
#         src = os.path.join( save_path, files )
#         if ind < 2000:
#             shutil.move( src, train_path )
#         else:
#             shutil.move( src, test_path )
#     except:
#         continue